In [1]:
import time
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain,LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
persist_directory = "../dbv2"
embeddings = OpenAIEmbeddings()
faqdb = Chroma(
    persist_directory=persist_directory,
    embedding_function = embeddings
)

Using embedded DuckDB with persistence: data will be stored in: ../dbv2


In [3]:
from langchain.prompts.prompt import PromptTemplate
template = """"Tickit is a rewards programme brought to you by Dubai Holding, offering an effortless instant way to earn and redeem points across your favourite brands and destinations in the UAE. The programme offers a comprehensive rewards experience across diverse categories including the emirate’s largest and most iconic attractions and entertainment destinations, leading theme parks, thriving dining and hospitality, online and in-store shopping, transportation and more. Tickit is the first rewards programme to use card-linked technology to earn and redeem points.
You are a customer service agent answering user queries about Tickit using the document below. 
If you can't find the answer in the document, say you don't know the answer and propose him to rephrase his query with more details.

Context: 
{context}\n\n

Current conversation:
{chat_history}

Question:
{question}\n\n

Answer:
"""
 
faq_prompt= PromptTemplate.from_template(template)

In [4]:
question = "How do I redeem points?"
similar_docs = faqdb.similarity_search(question, k=3)
similar_docs

[Document(page_content='Where can I redeem my points?\nAll participating redemption outlets are posted on the Tickit app for your convenience.\n\n1. You can find all the places where you can redeem your points on the Tickit app. \n2. The Tickit app will show you all the locations where you can use your points. \n3. The Tickit app will provide you with a list of all the outlets where you can redeem your points.', metadata={'source': '../data/docs/27.txt'}),
 Document(page_content='Is there any possibility in redeeming the points in cash? \nThe points are only used to make direct payment through a virtual payment card directly from your account at the Tickit app.\n\n1. Unfortunately, you cannot redeem your points for cash. The points can only be used to make payments through the virtual payment card in the Tickit app.\n\n2. No, you cannot exchange your points for cash. The points are only valid for direct payments through the virtual payment card in the Tickit app.\n\n3. Sorry, you canno

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
from langchain.chains import LLMChain,ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm=llm,\
                                              retriever=faqdb.as_retriever(search_kwargs={"k": 5}),\
                                              qa_prompt=faq_prompt)

In [7]:
chat_history = []
query = "What is Tickit?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

Tickit is a first-of-its-kind rewards programme brought to you by Dubai Holding, offering an effortless instant way to earn and redeem points across your favourite brands and destinations in the UAE. The programme offers a comprehensive rewards experience across diverse categories including the emirate’s largest and most iconic attractions and entertainment destinations, leading theme parks, thriving dining and hospitality, online and in-store shopping, transportation and more.


In [8]:
chat_history = [(query, result["answer"])]
query = "Who owns it?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

Tickit is owned by Dubai Holding.


In [9]:
chat_history = [(query, result["answer"])]
query = "How to redeem points?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

You can redeem your points with Tickit by using your linked credit or debit card registered on the app. All participating redemption outlets are posted on the Tickit app for your convenience.


In [10]:
chat_history = [(query, result["answer"])]
query = "How far is moon from earth?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

I'm sorry, I don't know the answer to that question. Could you please rephrase your query with more details?
